# Spam detector using machine learning

hurry | offer | meeting | label (output)
 1    |  1    | 0       | Spam
 0    |  0    | 1       | Ham

1. Prepare data:
    -import data, lower case, tokenise data, remove stop words, stemming, feature extration
2. split the data into Training and Testing (80-20%)
3. x, y : Feature Extration
    - Get List of all words from all messages, Count the frequency, 
4. Choose the algorithem => Naive Bayes
5. Train the alorithm with the traing data. --> creates a model.
6. Test the Model with testing dataset.
7. Calculate Metric for Accuracy.
8. If accuracy is good release the model if not, add more methods in preparing dataset.
    

-Data Processing


In [146]:
import nltk
from nltk.tokenize import word_tokenize # object
from nltk.corpus import stopwords # object
from nltk.stem import SnowballStemmer # Class 
from nltk.stem import WordNetLemmatizer # Class

stemmer = SnowballStemmer('english')
st = stopwords.words('english')

## Practicing
wrds = ['Testing', 'bulish', 'fasioNizing', 'sunseting', 'is', 'u', 'the']
wrds = [stemmer.stem(word) for word in wrds if word not in st]
wrds


['test', 'bulish', 'fasion', 'sunset', 'u']

# Full data practice
## Preprocessing

In [147]:
# Full data practice
import pandas as pd

# Load the dataset into a DataFrame

df = pd.read_csv('SMSSpamCollection.txt',sep='\t', names = ['label', 'message'])
df = pd.DataFrame(df)
print("*Lenght of dataframe =",len(df))
print("\n*Index 0 befor shuffling data\n", df[:1])

# Shuffle the dataset
""" 
In the above example, we load the dataset from a CSV file
into a DataFrame called `df`. We then use the `sample()` 
function to shuffle the DataFrame by specifying `n=len(df)`,
which shuffles all the rows. 
Finally, we reset the index of the shuffled DataFrame using the `reset_index()` 
function with `drop=True` to remove the old index.
 """
df = df.sample(n=len(df))
print("\n-Index 0 after shuffling data. Notice the Index Number:\n ", df[:1])
# Reset the index of the shuffled DataFrame
df = df.reset_index(drop=True)
print("\n-Index 0 after Reset the index. Notice the Index Number:\n ", df[:1])

# Removing some data
ignore = [';','.','(',')',',''s','c',',','&','*','U','-','?','!','\d', '&',"'s",'...', ':', 'u','?' ,'..','2']

def preprocessing(document):
    document = document.lower()
    words = word_tokenize(document)
    words = [word for word in words if word not in st] # remove stopwords
    words = [stemmer.stem(word) for word in words] # Stemming
    words = [word for word in words if word not in ignore] # RegEx to remove words not part of \w| [a-zA-Z0-9_]     
    return words

# Converting the document in below format- List of tuples of list
# [([w1,w2,w3], label1), ([w4,w5], label2),... ]

dataset = []
for index, row in df.iterrows():
    dataset.append((preprocessing(row['message']), row['label']))
print(dataset)



*Lenght of dataframe = 5572

*Index 0 befor shuffling data
   label                                            message
0   ham  Go until jurong point, crazy.. Available only ...

-Index 0 after shuffling data. Notice the Index Number:
       label                                            message
2798   ham  If I get there before you after your ten billi...

-Index 0 after Reset the index. Notice the Index Number:
    label                                            message
0   ham  If I get there before you after your ten billi...
[(['get', 'ten', 'billion', 'call', 'text', 'help', 'god'], 'ham'), (['ok.', 'later', 'show', 'around', '8-8:30', 'want', '+', 'cld', 'drink', 'wld', 'prefer', 'spend', 'money', 'nosh', "n't", 'mind', 'nxt', 'wk'], 'ham'), (['import', 'inform', '4', 'orang', 'user', '0796xxxxxx', 'today', 'ur', 'lucki', 'day', 'find', 'log', 'onto', 'http', '//www.urawinner.com', 'fantast', 'prizeawait'], 'spam'), (['live', 'simpl', 'love', 'also', 'simpl', 'laugh', 'simpl'

## Split the data into Training and Testing
- 80 to 20 Ratio

In [148]:
slice_index = int(len(dataset)*.80) # Output is float, Converting to int
len(dataset), slice_index

(5572, 4457)

In [149]:
train_messages = dataset[:slice_index]
test_messages = dataset[slice_index:]
print(len(train_messages))
print(len(test_messages))

4457
1115


# Feature Extraction

- Vocabulary 
- SMS1 : F, F, F.....T

-Clubbing all messages into one list

In [150]:

allwords = []
for (message, label) in dataset:
    allwords.extend(message)
len(allwords)



52868

-Counting the frequency of each word

In [151]:
word_freq = nltk.FreqDist(allwords)
word_freq

FreqDist({'call': 658, 'go': 455, 'get': 447, "'m": 395, 'ur': 391, "n't": 366, 'gt': 318, 'lt': 316, 'come': 303, '4': 289, ...})

-Removing Dublicates words in list.

In [165]:
word_features = list(word_freq.keys()) # gives dict, convert to list
type(word_features), len(word_features)


(list, 7963)

- Extracting the features 
- Converting them in below format
[({'contains(go)': False, 'contains(jurong)': False, 'contains(point)': False, 'contains(,)': False, 'contains(crazy)': False, 'contains(..)': False, 'contains(available)': False, 'contains(bugis)': False, 'contains(n)': False, 'contains(great)': False, 'contains(world)': False, 'contains(la)': False, 'contains(e)': False, 'contains(buffet)': False, 'contains(...)': False, 'contains(cine)': False, 'contains(get)': False, 'contains(amore)': False, 'contains(wat)': False, 'contains(ok)': False, 'contains(lar)': False})]

In [179]:
# converting the all words in the list to dict with boolen values.
def extract_features(message):
    feature = {} 
    for word in word_features:
        feature[f'contains({word})'] = word in message
    return feature

training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features, test_messages)
pd.DataFrame(training_set[:1])
    
    

,0,1
0,"{'contains(get)': True, 'contains(ten)': True,...",ham


-Training Data with Naive Bayes Algorithm

In [181]:
model = nltk.NaiveBayesClassifier.train(training_set)

In [182]:
nltk.classify.accuracy(model, testing_set)

0.9775784753363229

In [184]:
test_message = 'CONGRATULATIONS!! As a valued account holder you have been selected to receive a £900 prize reward! Valid 12 hours only.'
test_features = extract_features(preprocessing(test_message))
model.classify(test_features)


'spam'